# Text Generation with Neural Networks
In this notebook our objective is to demonstrate how to generate text using a character-based RNN working with a dataset of Shakespeare's  writing

# Notebook Overview
- Imports Dependencies
- Define Constants and Paths and Configure Logging
- Generating Predictions

## Imports Dependencies 

In [ ]:
# Standard Library Imports
import logging

# Third-Party Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU, InputLayer
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard


# Define Constants and Paths and Configure Logging

In [ ]:
# Define global experiment and run names to be used throughout the notebook
MODEL_NAME = "tf_rnn_model.h5"

# Set up the paths
DATA_PATH = "shakespeare.txt"
TENSORBOARD_PATH = "/phoenix/tensorboard/tensorlogs"


# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [ ]:
# Configure the logging module with desired format and level
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create a logger for this notebook
logger = logging.getLogger('text-generation-TF-notebook')
logger.info("Logging configured successfully")

## Get Text Data

This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [ ]:
path_to_file = DATA_PATH
text = open(path_to_file, 'r').read()

In [ ]:
logger.info('First 600 chars: \n')
logger.info(text[:600])

## Preparing textual data

We need to encode our data to give the model a proper numerical representation of our text.

In [ ]:
# creates a set of unique characters found in the text
vocab = sorted(set(text))

### Text Vectorization

In [ ]:
char_to_int = {u:i for i, u in enumerate(vocab)}
# assigns a unique integer to each character in a dictionary format, 
# creating a mapping that can later be used to transform encoded predictions back into characters

In [ ]:
int_to_char = np.array(vocab)
# reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers, which is used to encode the text.

In [ ]:
encoded_text = np.array([char_to_int[c] for c in text])
# encodes the entire text as an array of integers, with each integer representing the character at that position
# in the text according to the encoder dictionary

## Creating Training Batches

Training batches are a way of dividing the dataset into smaller, manageable groups of data points that are fed into a machine learning model during the training process.

In [ ]:
seq_len = 120 # length of sequence for a training example
total_num_seq = len(text)//(seq_len+1) # total number of training examples

# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [ ]:
def create_seq_targets(seq):
    """
    Function that takes a sequence as input, duplicates, and shifts it to align the input and label. 

    Args:
        seq: sequence of characters

    Returns:
        The text input and corresponding target.
    """
    try:
        input_txt = seq[:-1]
        target_txt = seq[1:]
        return input_txt, target_txt
    except Exception as e:
            logger.error(f"Error creating sequences of targets: {str(e)}")

In [ ]:
dataset = sequences.map(create_seq_targets)

In [ ]:
# Batch size
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## Creating the GRU Model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units
rnn_neurons = 1026

In [ ]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    """Architecture to create the model.

    Args:
        vocab_size: Length of the vocabulary in chars.
        embed_dim: Embedding dimension.
        rnn_neurons: Number of RNN units.
        batch_size: Size of the batchs.

    Returns:
        Model.
    """
    try:
        model = Sequential()
        model.add(InputLayer(batch_shape=(batch_size, None)))
        
        model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim))

        model.add(GRU(rnn_neurons,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))

        model.add(Dense(vocab_size))
        model.compile(optimizer='adam', loss=sparse_cat_loss)
        logger.info("Model architecture created successfully")
        return model
    except Exception as e:
            logger.error(f"Error creating model architecture: {str(e)}")

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size)
model.summary()


## Instance of the Model

In [ ]:
model = create_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    rnn_neurons=rnn_neurons,
    batch_size=batch_size
)


In [ ]:
# TensorBoard
log_dir = TENSORBOARD_PATH
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## Training the model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  logger.info(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
epochs = 20
model.fit(dataset,epochs=epochs, callbacks=[tensorboard_callback])

## Saving the Model

In [ ]:
model_name = MODEL_NAME

In [ ]:
model.save(f'models/{model_name}') 
logger.info("Model saved")

## Load Model

In [ ]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights(f'models/{model_name}')

model.build(tf.TensorShape([1, None]))

# Generating Predictions

In [ ]:
def generate_text(model, start_seed="The ", gen_size=100, temp=1.0):
    """
    Generates a sequence of text using the trained character-level language model.

    Args:
        model: Model created on function create_model
        start_seed: Set of characters that will be the beginning of the text. 
        gen_size : Number of characters. Defaults to 100.
        temp: Controls the randomness of the predictions made by the model.

    Returns:
        The full generated text including the seed and the newly predicted characters.
    """
    try:
        num_generate = gen_size
        input_eval = [char_to_int[s] for s in start_seed]
        input_eval = tf.expand_dims(input_eval, 0)
        text_generated = []
        temperature = temp


        for i in range(num_generate):
            predictions = model(input_eval)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(int_to_char[predicted_id])

        return start_seed + ''.join(text_generated)
    except Exception as e:
            logger.error(f"Error making predictions: {str(e)}")


#### Generating a text with 1000 chars starting with word 'Confidence'

In [ ]:
for layer in model.layers:
    if hasattr(layer, 'reset_states'):
        layer.reset_states()
print(generate_text(model, start_seed="Confidence ", gen_size=1000))

#### Generating a text with 1000 chars starting with word 'Love'

In [ ]:
print(generate_text(model, start_seed="Love ", gen_size=1000))

Built with ❤️ using Z by HP AI Studio.